In [1]:
from utils.long_residuals_mlp import *
from utils.utils import *
import cv2
from sklearn.svm import SVC


2024-01-14 15:29:22.794837: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-14 15:29:22.840272: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 15:29:22.840340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 15:29:22.841436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 15:29:22.847657: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-14 15:29:22.848088: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Reshape

PATCH_SIZE = 32
CLASSES      = ['coast','forest','highway','inside_city','mountain','Opencountry','street','tallbuilding']

MODEL_WEIGHTS = '/home/georg/projects/university/C3_ML_for_CV/project23-24-04/Task2/long_residuals-4096-32-1e-05/long_residuals-4096-32-1e-05_best.h5'
images_directory = '/home/georg/projects/university/C3_ML_for_CV/MIT_split/'


model = build_long_residuals_mlp(input_shape=(PATCH_SIZE, PATCH_SIZE, 3),
                                  activation='relu', 
                                  regularization_coeff=0.00001, 
                                  num_units=4096, 
                                  filename='myMLP.png')

model.load_weights(MODEL_WEIGHTS)

model_layer = Model(inputs=model.input, outputs=model.get_layer('dense_38').output)



for subdir1 in os.listdir(images_directory):
    for subdir2 in os.listdir(os.path.join(images_directory, subdir1)):
        for file in os.listdir(os.path.join(images_directory, subdir1, subdir2)):
            img = load_and_preprocess_image(image_path=os.path.join(images_directory, subdir1, subdir2, file),
                                            target_size=(32, 32,3))
            
            layer_output = model_layer.predict(img)
            layer_output = np.array(layer_output)
            print(layer_output.shape)
            # save np array to csv file
            np.savetxt(os.path.join('/home/georg/projects/university/C3_ML_for_CV/project23-24-04/Task2/embeddings/image', file.split('.')[0]+'.csv'), layer_output, delimiter=',')

In [25]:
EMBEDDINGS_DIR = '/home/georg/projects/university/C3_ML_for_CV/project23-24-04/Task2/embeddings/image'
PATCH_SIZE = 32

SVM_C = 1e-3
SVM_KERNEL = 'linear'
SVM_GAMMA = 'scale'


In [7]:
embeddings_dict = read_embeddings(directory=EMBEDDINGS_DIR)

training_files, training_labels = load_data_from_directory('/home/georg/projects/university/C3_ML_for_CV/MIT_split/train/')
test_files, test_labels = load_data_from_directory('/home/georg/projects/university/C3_ML_for_CV/MIT_split/test/')


training_embeddings = np.array([embeddings_dict[os.path.basename(file)] for file in training_files])
test_embeddings = np.array([embeddings_dict[os.path.basename(file)] for file in test_files])

print(training_embeddings.shape)
print(test_embeddings.shape)

(1881, 256)
(807, 256)


In [26]:
print(f'Fitting svm with C={SVM_C}, kernel={SVM_KERNEL}, gamma={SVM_GAMMA}')
svm = SVC(C=SVM_C, kernel=SVM_KERNEL, gamma=SVM_GAMMA)
svm.fit(training_embeddings, training_labels)
# predict labels for test data
pred_labels = svm.predict(test_embeddings)

# get accuracy metric for current run
print(get_metrics(test_labels, pred_labels))

Fitting svm with C=0.001, kernel=linear, gamma=scale
(0.4250309789343247, 0.41030224034343876, 0.4946766485828986, 0.3904486249117256)


/home/georg/projects/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
